In [1]:
# Dependencies and Setup
import base64
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import base64
from requests.auth import HTTPBasicAuth
import scipy.stats as st

# Import Spotify API key
from api_keys import spotify_client_id, spotify_client_secret

# Spotify Playlists Data Output File (CSV)
spotify_playlists_csv = "../datasets/spotify_playlists_data.csv"

In [3]:
"""
Code to manage Spotify's Client Credentials Flow as described here:
https://developer.spotify.com/documentation/general/guides/authorization-guide/#client-credentials-flow
"""
# Build the Spotify Client ID and Secret combination
spotify_client_id_secret = spotify_client_id + ":" + spotify_client_secret
# Generate the base64 encoded string that contains the client ID and client secret key
base64_encoding = base64.b64encode(spotify_client_id_secret.encode()).decode()

# Build the Spotify API URL to get the Bearer token
spotify_token_url = 'https://accounts.spotify.com/api/token'

# Build the payload, i.e. the request body parameters that need to be passed to the token API URL
payload = 'grant_type=client_credentials'
# Build the header for Content-Type and Basic Auth containing the base64 encoded string so we can generate the Spotify API token
token_api_headers = { 'Content-Type': 'application/x-www-form-urlencoded', 'Authorization': 'Basic %s' % base64_encoding }

# Call the token API URL and save as a JSON response object
spotify_token_response = requests.post(spotify_token_url, headers=token_api_headers, data=payload).json()
# Retrieve the Bearer token from the response object
spotify_access_token = spotify_token_response['access_token']

,href,items,limit,next,offset,previous,total
0,https://api.spotify.com/v1/playlists/37i9dQZF1...,"{'added_at': '2020-01-08T21:58:16Z', 'added_by...",100,https://api.spotify.com/v1/playlists/37i9dQZF1...,0,None,184
1,https://api.spotify.com/v1/playlists/37i9dQZF1...,"{'added_at': '2020-01-08T21:58:16Z', 'added_by...",100,https://api.spotify.com/v1/playlists/37i9dQZF1...,0,None,184
2,https://api.spotify.com/v1/playlists/37i9dQZF1...,"{'added_at': '2020-01-08T21:58:16Z', 'added_by...",100,https://api.spotify.com/v1/playlists/37i9dQZF1...,0,None,184
3,https://api.spotify.com/v1/playlists/37i9dQZF1...,"{'added_at': '2020-01-08T21:58:16Z', 'added_by...",100,https://api.spotify.com/v1/playlists/37i9dQZF1...,0,None,184
4,https://api.spotify.com/v1/playlists/37i9dQZF1...,"{'added_at': '2020-01-08T21:58:16Z', 'added_by...",100,https://api.spotify.com/v1/playlists/37i9dQZF1...,0,None,184


In [11]:
"""
Code to retrieve a Spotify Playlist's Tracks and related data:
https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlists-tracks/
"""
# Sample Playlist ID
spotify_playlist_id = '37i9dQZF1DWZeKCadgRdKQ'

# Build Spotify's Get Playlist's Tracks URL
spotify_playlists_url = f'https://api.spotify.com/v1/playlists/{spotify_playlist_id}/tracks'
# Build the header for Bearer token containing the base64 encoded string so we can retrieve the playlist's tracks 
playlist_api_headers = { 'Authorization': 'Bearer %s' % spotify_access_token }
# Call the Get Playlist's Tracks API URL and save as a JSON response object
spotify_playlists_response = requests.get(spotify_playlists_url, headers=playlist_api_headers, data={}).json()

# Initialize lists to save playlist data
track_added_at = []
track_names = []
track_popularity = []
track_duration = []

# Get total tracks in playlist
total_tracks = spotify_playlists_response['total']

# Playlist returns a track object
for item in range(len(spotify_playlists_response['items'])):
    # Get the date and time the track was added to the playlist 
    track_added_at.append(spotify_playlists_response['items'][item]['added_at'])
    # Get the name of the track
    track_names.append(spotify_playlists_response['items'][item]['track']['name'])
    # Get the popularity score of the track
    track_popularity.append(spotify_playlists_response['items'][item]['track']['popularity'])
    # Get the track duration (song length) in milliseconds
    track_duration.append(spotify_playlists_response['items'][item]['track']['duration_ms'])

# Save the retrieved weather data into a Data Frame
playlist_data_df = pd.DataFrame( {'Track Name': track_names, 'Track Added At': track_added_at, 'Track Popularity': track_popularity, 'Track Duration': track_duration } )

# Display the weather data frame
playlist_data_df

,Track Name,Track Added At,Track Popularity,Track Duration
0,Under The Wind,2020-01-08T21:58:16Z,60,240000
1,Safe And Sound,2020-01-08T21:58:16Z,61,169363
2,alpha waves,2020-01-08T21:58:16Z,57,205127
3,Dawning,2020-01-08T21:58:16Z,60,206419
4,Curiosity,2020-01-08T21:58:16Z,61,155820
...,...,...,...,...
95,Tracking Aeroplanes,2020-01-08T21:58:16Z,53,468927
96,Final Mid,2020-01-08T21:58:16Z,50,129199
97,they sink,2020-01-08T21:58:16Z,54,155484
98,Ether,2020-01-08T21:58:16Z,51,189040
